# CModel Analysis and Testing

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import sdfec_cmodel

In [2]:
with open("data/data.vcd", "r") as f:
    lines = f.read().split("\n")
    data = [int(v[1:], 2) for v in lines if len(v) > 0]

In [3]:
block = 15711630
din_beats = 372
blocks = blocks = len(data) // din_beats

In [4]:
def split_vector(v, count, l):
    out = []
    mask = (1 << l)-1
    
    for i in range(count):
        out.append(v & mask)
        v >>= l
    
    return np.array(out, dtype=np.int8).astype(float) / 4.0
    

In [5]:
din_sim = np.array([split_vector(v, 16, 8) for v in data])
din_sim = din_sim.reshape(din_sim.size)

In [6]:
model = sdfec_cmodel.sdfec_core("sdfec", "docsis_31_medium.yml")

sdfec NOTE: Initializing LDPC code definitions from file...
sdfec NOTE: Code number: 0 -> no_id 


In [7]:
ber = sdfec_cmodel.ldpc_ber_tester(0, model)

In [8]:
ber.snr_scale = 4
ber.snr = 0

In [9]:
din_mod_raw,status,_,dout = ber.simulate_block(block, max_iterations=32, term_on_no_change=False)

In [10]:
din_mod = np.array(din_mod_raw)

In [11]:
din_sim[:128]

array([-4.25, -7.75, -1.5 ,  0.25,  0.25, -7.75, -3.  , -7.75,  2.75,
       -4.25, -1.  ,  0.75, -0.75, -1.  , -7.5 , -2.75, -5.5 ,  0.25,
       -5.75, -5.75,  4.  ,  0.25,  2.25, -0.25, -1.75, -5.25,  2.  ,
       -5.  , -3.75, -1.  , -7.25, -4.75, -4.5 , -0.5 , -7.75, -5.5 ,
       -1.5 , -1.5 , -1.  ,  0.5 , -1.  , -7.75, -2.75, -4.5 , -0.25,
       -6.75,  0.25,  4.75, -7.75, -3.75,  1.5 , -6.  , -7.25, -7.75,
       -0.75,  2.25,  0.75,  3.75,  0.5 , -4.75,  6.  , -3.75, -2.25,
       -3.25, -7.75,  1.25,  4.  ,  3.  ,  1.75, -4.25, -7.75, -1.5 ,
       -4.5 , -6.5 , -5.75, -7.  , -7.75, -3.5 , -4.25, -7.75, -7.5 ,
       -0.5 , -7.25, -7.75, -7.75,  0.75, -6.  ,  0.25, -4.75, -0.75,
       -7.75, -0.75, -3.75,  1.75,  5.25, -1.25,  3.25, -2.25, -0.75,
       -7.25,  3.75, -5.  , -2.5 , -7.75, -0.25, -7.75, -3.25, -6.25,
        0.5 , -7.75, -5.25, -5.  , -7.75, -2.5 , -3.75, -7.5 , -0.75,
        2.5 , -1.25, -5.75, -2.25, -4.75, -7.25, -0.5 , -4.5 , -4.25,
       -6.75, -1.5 ]

In [12]:
din_mod[:128]

array([-4.25, -7.75, -1.5 ,  0.25,  0.25, -7.75, -3.  , -7.75,  2.75,
       -4.25, -1.  ,  0.75, -0.75, -1.  , -7.5 , -2.75, -5.5 ,  0.25,
       -5.75, -5.75,  4.  ,  0.25,  2.25, -0.25, -1.75, -5.25,  2.  ,
       -5.  , -3.75, -1.  , -7.25, -4.75, -4.5 , -0.5 , -7.75, -5.5 ,
       -1.5 , -1.5 , -1.  ,  0.5 , -1.  , -7.75, -2.75, -4.5 , -0.25,
       -6.75,  0.25,  4.75, -7.75, -3.75,  1.5 , -6.  , -7.25, -7.75,
       -0.75,  2.25,  0.75,  3.75,  0.5 , -4.75,  6.  , -3.75, -2.25,
       -3.25, -7.75,  1.25,  4.  ,  3.  ,  1.75, -4.25, -7.75, -1.5 ,
       -4.5 , -6.5 , -5.75, -7.  , -7.75, -3.5 , -4.25, -7.75, -7.5 ,
       -0.5 , -7.25, -7.75, -7.75,  0.75, -6.  ,  0.25, -4.75, -0.75,
       -7.75, -0.75, -3.75,  1.75,  5.25, -1.25,  3.25, -2.25, -0.75,
       -7.25,  3.75, -5.  , -2.5 , -7.75, -0.25, -7.75, -3.25, -6.25,
        0.5 , -7.75, -5.25, -5.  , -7.75, -2.5 , -3.75, -7.5 , -0.75,
        2.5 , -1.25, -5.75, -2.25, -4.75, -7.25, -0.5 , -4.5 , -4.25,
       -6.75, -1.5 ]

In [13]:
np.count_nonzero(din_sim[:5940] - din_mod)

0

In [14]:
rnd_vector = ber.get_rnd_vector(block, 0)

In [15]:
sdfec_cmodel.bitexact_gaussian(*rnd_vector[0])

(-0.0908203125, -1.0166015625)